# Install

In [ ]:
!pip install -r requirements.txt

# Import

In [6]:
from agentic_doc.parse import parse
from agentic_doc.utils import viz_parsed_document
from agentic_doc.config import VisualizationConfig
from bs4 import BeautifulSoup
import json
import pandas as pd
from io import StringIO
import os
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

from __future__ import annotations

from typing import List

from pydantic import BaseModel, Field
from agentic_doc.parse import parse
from agentic_doc.connectors import LocalConnectorConfig




In [7]:
# Read every img files in /picsToExtract
# Save Analyzed results in /picsFinished

# Store_list

In [8]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file("suksiri-purchase-test-0f09e84df6dd.json", scopes=SCOPES)
client = gspread.authorize(creds)
service = build("sheets", "v4", credentials=creds)

spreadsheet_id = "17chQLsKcpyZNnJyw8Ads-WRz45kNvI1AbvwsdIlcXqs"

sheets = client.open_by_key(spreadsheet_id)
store_data = sheets.worksheet("ข้อมูลร้านค้า")
sheet = sheets.worksheet("COPY รายการสินค้า")

table_range = "ข้อมูลร้านค้า!B2:G"  

result = service.spreadsheets().values().get(
    spreadsheetId=spreadsheet_id,
    range=table_range
).execute()

values = result.get("values", [])

# Convert to DataFrame
df_table2 = pd.DataFrame(values[1:], columns=values[0])  # first row is header

# --- Keep only rows where 'ร้านค้า' is not None or empty ---
df_filtered = df_table2[df_table2['ร้านค้า'].notna() & (df_table2['ร้านค้า'] != '')].copy()

# --- Convert 'ยังไม่รวม VAT' from string 'TRUE'/'FALSE' to boolean ---
df_filtered['ยังไม่รวม VAT'] = df_filtered['ยังไม่รวม VAT'].map(lambda x: True if str(x).upper() == 'TRUE' else False)


unique_store_list = df_filtered['ร้านค้า'].unique().tolist()

print(unique_store_list)

df_filtered.head()

file_cache is only supported with oauth2client<4.0.0 (__init__.py:49)
['จเร', 'ไทวัสดุ', 'ดีเดย์', 'CSS', 'โฮมโปร', 'H.T.', 'อิทธิฤทธิ์ ไนซ์', 'อนันตกาญจน์', 'เทียนทอง', 'บ้านสุขภัณฑ์', 'ภูเก็ต Tools', 'โมเดิร์น PVC', 'อินฟินิตี้ IT', 'KML', 'PEMCO', 'ซุปเปอร์เซ็นเตอร์', 'แสงนิรันดร์', 'แอดวานซ์', 'คอมพลีท อิเล็คทริเคิล โซลูชั่น', 'SMART HOME', 'พีแอลวาย', 'กิจเจริญ', 'สากลเวิร์ค', 'เดอะโฮมมาร์ทแม็ก', 'กี่หิ้น', 'จอมทองเคหะภัณฑ์', 'แสงชัย', 'ซุปเปอร์ชีป', 'DP', 'เอสซีที', 'อำพัน', 'องอาจ', 'โรงไม้ตะกั่วป่า', 'มินิเอเจอร์', 'ฉลองคอนกรีต', 'ลิกซิล', 'ไลน์คอร์นเนอร์', 'มัลติ ไอที เซอร์วิส', 'ธนากลาส', 'เกาะแก้วชัยสตีล', 'ควอลิตี้ โฮมมาร์ท', 'S.M.', 'ไลท์ติ้งดรีม', 'ภูเก็ต โชคโยธา', 'เบรน ไทย', 'Tooltopia', 'ราชาท่อใยหิน', 'สมถวิลการเกษตร', 'ดีพี โซลูชั่น', 'ทีซีเทคมาร์ค', 'ภูเก็ต อิงค์']


,ร้านค้า,ยังไม่รวม VAT,จำนวนสินค้า,จำนวนออเดอร์,มูลค่ารวม,ซื้อทุกๆกี่วัน
0,จเร,True,188,113,"487,772",2.46
1,ไทวัสดุ,False,92,57,"75,756",4.30
2,ดีเดย์,False,109,49,"87,439",4.42
3,CSS,True,65,39,"1,063,699",5.27
4,โฮมโปร,False,67,26,"32,569",8.04


# ExtractedDocumentFieldsSchema Class

In [19]:
class CompanyInfo(BaseModel):
    companyName: str = Field(
        ...,
        description=f'Identify the official name of the seller/supplier company as stated in the document. Compare it to the following list of known companies: [{", ".join(unique_store_list)}]. Return the closest match, ignoring common words like ["ห้างหุ้นส่วนจำกัด", "บริษัท", "จำกัด", "บจก.", "หจก.", "จํากัด", "ก้าวไกล"] for matching purposes. If no sufficiently similar match is found, retain the original text from the document.',
        title='Company Name',
    )
    taxId: str = Field(
        ...,
        description="The seller/supplier company's tax identification number.",
        title='Tax Identification Number',
    )


class CustomerInfo(BaseModel):
    customerName: str = Field(
        ..., description='The name of the customer or recipient.', title='Customer Name'
    )
    

class DocumentInfo(BaseModel):
    documentNumber: str = Field(
        ...,
        description='Unique identifier or reference number(เลขที่กำกับ) for the document.',
        title='Document Number',
    )
    documentDate: str = Field(
        ..., description='Date the document was issued with Year formatted in คริสต์ศักราช (ค.ศ.)/AD (Anno Domini) if it was initially written in the format of พุทธศักราช (พ.ศ.)/BE (Buddhist Era)', title='Document Date'
    )
    


class Item(BaseModel):
    description: str = Field(
        ..., description='Description of the item or service.', title='Description'
    )
    quantity: str = Field(..., description='Quantity of the item.', title='Quantity')
    unitPrice: str = Field(
        ..., description='Price per unit of the item.', title='Unit Price'
    )
    unitName: str = Field(..., description='Unit of measurement for the item, for example, meter, Pcs, ea, kg, box, อัน, ใบ, เส้น, ท่อน, ตัว, กระป๋อง, หลอด, ม้วน', title='Unit Name')
    amount: str = Field(..., description='Total amount for the item.', title='Amount')
    discount: str = Field(..., description='Discount applied to the item.', title='Item Discount')
    discountType: str = Field(..., description='Type of discount applied in Thai Baht or in Percentage or No Discount', title='Discount Type')

class Totals(BaseModel):
    grossAmount: str = Field(
        ...,
        description='Total gross amount before discounts and taxes.',
        title='Gross Amount',
    )
    netAmount: str = Field(
        ..., description='Net amount after discounts.', title='Net Amount'
    )
    vat: str = Field(..., description='Value-added tax amount.', title='VAT')
    grandTotal: str = Field(
        ...,
        description='Total amount payable including all taxes.',
        title='Grand Total',
    )


class ExtractedDocumentFieldsSchema(BaseModel):
    
    companyInfo: CompanyInfo = Field(
        ...,
        description='Key company details from headers and form fields.',
        title='Company Information',
    )
    customerInfo: CustomerInfo = Field(
        ...,
        description='Details about the customer or recipient of the invoice/delivery.',
        title='Customer Information',
    )
    documentInfo: DocumentInfo = Field(
        ...,
        description='Key identifiers and dates for the document.',
        title='Document Information',
    )
    items: List[Item] = Field(
        ...,
        description='List of items, products, or services from the main table(s) in the document.',
        title='Itemized Table',
    )
    totals: Totals = Field(
        ...,
        description='Summary of financial totals from the document.',
        title='Totals and Summary',
    )
    


# Execute

In [10]:
results = parse("./picsToExtract/1.jpeg", extraction_model=ExtractedDocumentFieldsSchema)
fields = results[0].extraction


2025-08-28 08:07:36 [info   ] API key is valid.              [agentic_doc.utils] (utils.py:42)
2025-08-28 08:07:36 [info   ] Parsing 1 documents            [agentic_doc.parse] (parse.py:280)


Parsing documents:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-08-28 08:08:20 [info   ] Time taken to successfully parse a document chunk: 44.15 seconds [agentic_doc.parse] (parse.py:825)


Parsing documents: 100%|██████████| 1/1 [00:44<00:00, 44.11s/it]


In [13]:
print("----------")
print(fields.companyInfo.companyName) 
## check company name in any of the list ## 
print(fields.companyInfo.taxId)

print("----------")
print(fields.customerInfo.customerName)

print("----------")
print(fields.documentInfo.documentNumber)
print(fields.documentInfo.documentDate)

print("----------")
print(fields.items[0].description)
print(fields.items[0].quantity)
print(fields.items[0].unitPrice)
print(fields.items[0].amount)
print(fields.items[0].discount)
print(fields.items[0].discountType)

print("----------")
print(fields.totals.grossAmount)
print(fields.totals.netAmount)
print(fields.totals.vat)
print(fields.totals.grandTotal)

# fields.companyInfo

----------
เกาะแก้วชัยสตีล
0833539000324
----------
บจก. สุขสิริ เอ็นจิเนียริ่ง
----------
HS68010419
2025-01-04
----------
เหล็กกล้าวาไนซ์ 4"(1.2-3.2มม.)
8
1650.00
13200.00
0.00
No Discount
----------
15030.00
14046.73
983.27
15030.00


# Build_rows

In [14]:
def build_rows(fields):
    metadata = {
        "วันเดือนปี": fields.documentInfo.documentDate,
        "ร้านค้า": fields.companyInfo.companyName,
        "เลขกำกับ": fields.documentInfo.documentNumber,
        # "taxId": fields.companyInfo.taxId,
        # "customerName": fields.customerInfo.customerName,
        # "grossAmount": fields.totals.grossAmount,
        # "netAmount": fields.totals.netAmount,
        # "vat": fields.totals.vat,
        # "grandTotal": fields.totals.grandTotal,
    }
    rows = []
    # Handle items
    for i, item in enumerate(fields.items):
        row = metadata.copy()
        row["รายการสินค้า"] = item.description
        row["จำนวน"] = float(item.quantity.replace(",", ""))
        row["หน่วย"] = item.unitName
        row["ราคาต่อหน่วย"] = float(item.unitPrice.replace(",", ""))
        
        # set defaults to empty cells
        row["ลดราคา(%)"], row["ลดราคา(บาท)"] = "", ""
        if item.discountType == "บาท":
            row["ลดราคา(บาท)"] = float(item.discount.strip('฿').replace(",", ""))
        elif item.discountType == "Percentage":
            row["ลดราคา(%)"] = float(item.discount.strip('%'))

        rows.append(row)
    
    return rows

build_rows(fields);
# {'วันเดือนปี': '2025-01-04',
#   'ร้านค้า': 'Communication & System Solution Public Company Limited',
#   'เลขกำกับ': 'IV-H680000099',
#   'รายการสินค้า': 'THW-A 25 SQ.MM. (ขาวตลอด) "BCC"',
#   'จำนวน': 200.0,
#   'หน่วย': 'Metre',
#   'ราคาต่อหน่วย': 25.44,
#   'ลดราคา(%)': 42.0,
#   'ลดราคา(บาท)': ''}

# GG Sheet

In [ ]:
def update_sheet(rows):
    df = pd.DataFrame(rows)
    values = df.values.tolist()
    
    start_row = len(sheet.get_all_values()) + 1  # +1 because Sheets are 1-indexed

    for i, row in enumerate(values):
        current_row = start_row + i

        # Column J: ยอดเงิน
        row.append("=Transactions_2[จำนวน]*Transactions_2[ราคาต่อหน่วย]")

        # Column K: ยอดเงินหลังลดราคา
        row.append("=IF(Transactions_2[ลดราคา(%)], Transactions_2[ยอดเงิน]*(1-Transactions_2[ลดราคา(%)]/100), Transactions_2[ยอดเงิน]-Transactions_2[ลดราคา(บาท)])")

        # Column L: ยอดรวมต่อรายการ
        row.append(f"=SUMIF(Transactions_2[เลขกำกับ], C{current_row}, Transactions_2[ยอดเงินหลังลดราคา])")

        # Column M: ยอดรวมหลังภาษี
        row.append(
            f"=IF(XLOOKUP(B{current_row}, Table2[ร้านค้า], Table2[ยังไม่รวม VAT], FALSE), $L{current_row}*1.07, $L{current_row})"
        )

    # Append all rows to the sheet
    sheet.append_rows(values, value_input_option="USER_ENTERED")

    return pd.DataFrame(values)
    
# Example usage
rows = build_rows(fields)
update_sheet(rows);


# Visualizatoon

In [ ]:
def visualize_parsing(filepath, parsed_doc):
    viz_config = VisualizationConfig(
        thickness=2,  # Thicker bounding boxes
        text_bg_opacity=0.8,  # More opaque text background
        font_scale=0.7,  # Larger text
        # color_map={
        #     ChunkType.TEXT: (255, 0, 0),  
        #     ChunkType.TABLE: (255, 255, 0), }
    )

    images = viz_parsed_document(
        filepath,
        parsed_doc,
        output_dir="picsFinished",
        viz_config=viz_config
    )
    

# Integration

In [20]:
config = LocalConnectorConfig()
results = parse(config, 
                connector_path="./picsToExtract", 
                connector_pattern="*.jpeg",
                extraction_model=ExtractedDocumentFieldsSchema
                )
for result in results:
    fields = result.extraction  
    rows = build_rows(fields)
    print(rows)
    update_sheet(rows)

    # visualize_parsing(filepath,results[0])


2025-08-28 08:19:16 [info   ] API key is valid.              [agentic_doc.utils] (utils.py:42)
2025-08-28 08:19:16 [info   ] Parsing 5 documents            [agentic_doc.parse] (parse.py:280)


Parsing documents:   0%|          | 0/5 [00:00<?, ?it/s]

HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-08-28 08:19:48 [info   ] Time taken to successfully parse a document chunk: 32.25 seconds [agentic_doc.parse] (parse.py:825)
HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-08-28 08:19:57 [info   ] Time taken to successfully parse a document chunk: 40.95 seconds [agentic_doc.parse] (parse.py:825)


Parsing documents:  20%|██        | 1/5 [00:40<02:43, 40.96s/it]

HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-08-28 08:20:08 [info   ] Time taken to successfully parse a document chunk: 52.40 seconds [agentic_doc.parse] (parse.py:825)
HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-08-28 08:20:15 [info   ] Time taken to successfully parse a document chunk: 27.22 seconds [agentic_doc.parse] (parse.py:825)
HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-08-28 08:21:48 [info   ] Time taken to successfully parse a document chunk: 152.55 seconds [agentic_doc.parse] (parse.py:825)


Parsing documents: 100%|██████████| 5/5 [02:32<00:00, 30.51s/it]

[{'วันเดือนปี': '2025-01-04', 'ร้านค้า': 'ห้างหุ้นส่วนจำกัด เกาะแก้วชัยสตีล', 'เลขกำกับ': 'HS68010419', 'รายการสินค้า': 'เหล็กกล้าวาไนซ์ 4"(1.2-3.2มม.)', 'จำนวน': 8.0, 'หน่วย': 'เส้น', 'ราคาต่อหน่วย': 1650.0, 'ลดราคา(%)': '', 'ลดราคา(บาท)': ''}, {'วันเดือนปี': '2025-01-04', 'ร้านค้า': 'ห้างหุ้นส่วนจำกัด เกาะแก้วชัยสตีล', 'เลขกำกับ': 'HS68010419', 'รายการสินค้า': 'เหล็กดำ 4"(1/4-1/2")', 'จำนวน': 1.0, 'หน่วย': 'เส้น', 'ราคาต่อหน่วย': 1830.0, 'ลดราคา(%)': '', 'ลดราคา(บาท)': ''}]


[{'วันเดือนปี': '2025-01-08', 'ร้านค้า': 'กิจเจริญ ก้าวไกล', 'เลขกำกับ': 'HS6801005', 'รายการสินค้า': '10-PPR-10120/ ท่อPPR D-20 เขียว/ขาว PN20(25:มัด)', 'จำนวน': 8.0, 'หน่วย': 'เส้น', 'ราคาต่อหน่วย': 216.0, 'ลดราคา(%)': 57.0, 'ลดราคา(บาท)': ''}, {'วันเดือนปี': '2025-01-08', 'ร้านค้า': 'กิจเจริญ ก้าวไกล', 'เลขกำกับ': 'HS6801005', 'รายการสินค้า': '10-PPR-10425/ ข้อ90PPR-D-25', 'จำนวน': 10.0, 'หน่วย': 'ตัว', 'ราคาต่อหน่วย': 22.0, 'ลดราคา(%)': 52.0, 'ลดราคา(บาท)': ''}]
[{'วันเดือนปี': '2025-01-11', 'ร้านค้า': 'ดีเดย์', 'เลขกำกับ': 'HS6800260', 'รายการสินค้า': 'ดอกสกัดไร้ครัว แกนกลม หัวแผน PUMPKIN(196115) 14mm', 'จำนวน': 2.0, 'หน่วย': 'ชิ้น', 'ราคาต่อหน่วย': 85.0, 'ลดราคา(%)': '', 'ลดราคา(บาท)': ''}, {'วันเดือนปี': '2025-01-11', 'ร้านค้า': 'ดีเดย์', 'เลขกำกับ': 'HS6800260', 'รายการสินค้า': 'ดอกสกัดไร้ครัว แกนกลม หัวแบน PUMPKIN (196108) 14mm', 'จำนวน': 2.0, 'หน่วย': 'ชิ้น', 'ราคาต่อหน่วย': 85.0, 'ลดราคา(%)': '', 'ลดราคา(บาท)': ''}, {'วันเดือนปี': '2025-01-11', 'ร้านค้า': 'ดีเดย์', 'เลขกำกับ